In [1]:
import importlib
from tqdm import tqdm
from tqdm import trange

import pentago
from pentago import agent, memory, minimax, learning_agent, neural_model, monte_carlo, evaluate, game, multi

import ray
ray.init()

In [3]:
initial_view = memory.MemoryView()
controller = pentago.Controller([minimax.MinimaxSearchAgent(depth=2)] * 2, initial_view)
for i in tqdm(range(1000)):
    controller.play_game()

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 1/1000 [00:03<56:22,  3.39s/it]

  0%|          | 2/1000 [00:04<46:44,  2.81s/it]

  0%|          | 3/1000 [00:06<40:03,  2.41s/it]

  0%|          | 4/1000 [00:07<35:25,  2.13s/it]

...

100%|█████████▉| 996/1000 [23:38<00:05,  1.41s/it]

100%|█████████▉| 997/1000 [23:40<00:04,  1.42s/it]

100%|█████████▉| 998/1000 [23:41<00:02,  1.42s/it]

100%|█████████▉| 999/1000 [23:42<00:01,  1.43s/it]

100%|██████████| 1000/1000 [23:44<00:00,  1.42s/it]

In [3]:
import dill as pickle

In [34]:
pickle.dump(initial_view, open('initial_view.pkl', mode='wb'))

In [4]:
initial_view = pickle.load(open('initial_view.pkl', mode='rb'))

In [2]:
importlib.reload(neural_model)
importlib.reload(learning_agent)
importlib.reload(game)
importlib.reload(monte_carlo)
model_params = {'dense_regularization_const': 1e-2,
                'regularization_const': 1e-4,
                'num_layers': 5,
                'kernel_size': 5,
                'num_filters': 64}
current_agent = learning_agent.NeuralAgent(model_params=model_params)
best_agent = learning_agent.NeuralAgent(model_params=model_params)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
print(evaluate.tally_wins([agent.RandomAgent(), neural_agent], trials=10))
print(evaluate.tally_wins([minimax.MinimaxSearchAgent(), neural_agent], trials=10, verbose=True))

In [ ]:
neural_agent.fit(initial_view.get_experiences(), epochs=10)

In [28]:
print(evaluate.tally_wins([agent.RandomAgent(), neural_agent], trials=10))
print(evaluate.tally_wins([minimax.MinimaxSearchAgent(), neural_agent], trials=10))

{0: 0, 1: 0, -1: 10}


{0: 0, 1: 9, -1: 1}


In [3]:
view = memory.MemoryView()
round = 0

In [ ]:
while True:
    round += 1
    multi.self_play_games(current_agent,
                          view,
                          30,
                          verbose=True,
                          num_workers=3)
    for e in range(4):
        current_agent.fit(view.get_experiences(sample_k=1000), epochs=1)
    if round % 5 == 0:
        wins = multi.tally_wins([minimax.MinimaxSearchAgent(), current_agent], 10, num_workers=3)
        print('Round {}, against minimax: {}'.format(round, wins))
        wins = multi.tally_wins([best_agent, current_agent], 10, num_workers=3)
        print('Round {}, against current best neural: {}'.format(round, wins))
        if wins[-1] > 7:
            print('Ratcheting best agent')
            best_agent.set_weights(current_agent.get_weights())

In [5]:
view.temp_memory.clear()